# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770783275770                   -0.53    8.0
  2   -2.772133785020       -2.87       -1.30    1.0
  3   -2.772169296027       -4.45       -2.63    1.0
  4   -2.772170718607       -5.85       -3.83    2.0
  5   -2.772170722846       -8.37       -4.40    2.0
  6   -2.772170723009       -9.79       -5.60    1.0
  7   -2.772170723015      -11.20       -6.00    2.0
  8   -2.772170723015      -13.01       -6.60    1.0
  9   -2.772170723015      -13.70       -7.27    2.0
 10   -2.772170723015      -14.18       -7.97    1.0
 11   -2.772170723015   +  -13.91       -8.64    2.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770748050153                   -0.53    8.0
  2   -2.772053063272       -2.88       -1.31    1.0
  3   -2.772082609035       -4.53       -2.55    1.0
  4   -2.772083399043       -6.10       -3.50 

1.773558062396603

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770910078195                   -0.53    9.0
  2   -2.772061922561       -2.94       -1.32    1.0
  3   -2.772083151598       -4.67       -2.52    1.0
  4   -2.772083365739       -6.67       -3.31    1.0
  5   -2.772083414684       -7.31       -3.78    2.0
  6   -2.772083417725       -8.52       -4.89    1.0
  7   -2.772083417809      -10.07       -5.36    2.0
  8   -2.772083417810      -11.91       -6.18    1.0
  9   -2.772083417811      -12.66       -6.74    2.0
 10   -2.772083417811      -13.91       -7.52    1.0
 11   -2.772083417811      -14.10       -8.05    2.0

Polarizability via ForwardDiff:       1.772534962252541
Polarizability via finite difference: 1.773558062396603
